<a href="https://colab.research.google.com/drive/1Qa9KWk0upKcLBqCwP66z6nzLP4TDnXE-?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### ART: Automatic Reasoning and Tool-use Agent

An agent that automatically selects and executes tools through:
1. Parse task library for similar tasks
2. Select relevant tool demonstrations
3. Generate reasoning steps
4. Execute actual tool calls at appropriate steps
5. Adapt based on tool execution results

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass
import re
from datetime import datetime

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [3]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [4]:
genai.configure(api_key=API_KEY)

In [8]:
class ARTAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.tools = {}
        self.task_library = []

    def add_tool(self, name, func, description, param_name):
        self.tools[name] = {"func": func, "desc": description, "param": param_name}

    def add_example(self, task, tools_used):
        """Add successful task example"""
        self.task_library.append({"task": task, "tools": tools_used})

    def find_similar(self, query):
        """Find similar past tasks"""
        if not self.task_library:
            return []

        # Simple keyword matching
        words = set(query.lower().split())
        similar = []
        for ex in self.task_library:
            ex_words = set(ex["task"].lower().split())
            if words & ex_words:  # Any overlap
                similar.append(ex)
        return similar[:2]

    def plan(self, query, similar):
        """Generate step-by-step plan"""
        tools_list = "\n".join([
            f"{n}({t['param']}): {t['desc']}"
            for n, t in self.tools.items()
        ])

        examples = "\n".join([
            f"'{ex['task']}' used: {', '.join(ex['tools'])}"
            for ex in similar
        ]) if similar else "No examples."

        prompt = f"""Create a step-by-step plan for this task.

        Task: {query}

        Tools (use exact parameter names):
        {tools_list}

        Similar tasks:
        {examples}

        Format each step as:
        Step N: description [TOOL: name(param="value")] or [TOOL: None]
        Use ONLY the parameter names shown above.

        Plan:"""

        return self.model.generate_content(prompt).text

    def execute(self, query):
        """Execute task with automatic tool selection"""
        print(f"\n🎯 Task: {query}\n")

        # Find similar tasks
        similar = self.find_similar(query)
        if similar:
            print(f"📚 Found {len(similar)} similar tasks\n")

        # Generate plan
        plan_text = self.plan(query, similar)
        print(f"📋 Plan:\n{plan_text}\n")

        # Execute plan
        tools_used = []
        results = []

        for line in plan_text.split("\n"):
            if not line.strip().startswith("Step"):
                continue

            print(f"▶ {line.split(':')[0]}")

            # Check for tool call
            tool_match = re.search(r'\[TOOL:\s*(\w+)\((.*?)\)\]', line)
            if tool_match:
                tool_name = tool_match.group(1)

                if tool_name == "None":
                    print("  (reasoning only)\n")
                    continue

                # Parse params - take first param only
                param_name = self.tools[tool_name]["param"]
                param_value = None

                param_match = re.search(r'\w+=["\'](.*?)["\']', tool_match.group(2))
                if param_match:
                    param_value = param_match.group(1)

                # Execute with correct param name
                try:
                    if param_value:
                        result = self.tools[tool_name]["func"](**{param_name: param_value})
                    else:
                        result = self.tools[tool_name]["func"]()
                    print(f"  ✅ {tool_name}: {result}\n")
                    results.append(result)
                    tools_used.append(tool_name)
                except Exception as e:
                    print(f"  ❌ Error: {e}\n")

        # Learn from success
        if results:
            self.add_example(query, tools_used)
            print(f"✅ Completed. Used: {', '.join(tools_used)}\n")

        return results

In [9]:
# Define simple tools
def fetch(source):
    data = {
        "sales": {"revenue": 150000, "units": 1200},
        "customers": {"total": 5000, "active": 3500},
    }
    return data.get(source.lower(), {})

def calc(expr):
    return eval(expr)

def analyze(data):
    return f"Analyzed: {len(data)} metrics"

def report(msg):
    return f"Report: {msg}"

In [10]:
# Usage
agent = ARTAgent()

agent.add_tool("fetch", fetch, "Get data from source", "source")
agent.add_tool("calc", calc, "Calculate expression", "expr")
agent.add_tool("analyze", analyze, "Analyze data", "data")
agent.add_tool("report", report, "Create report", "msg")

# Add examples
agent.add_example("analyze sales", ["fetch", "analyze", "report"])

# Run
agent.execute("Get sales data and calculate total revenue")
agent.execute("Analyze customer data")
agent.execute("Get sales and create report")  # Should use learned pattern

print(f"📚 Learned {len(agent.task_library)} tasks")


🎯 Task: Get sales data and calculate total revenue

📚 Found 1 similar tasks

📋 Plan:
Okay, here's a step-by-step plan to get sales data and calculate total revenue, following the provided format and constraints:

**Plan:**

**Step 1**: Fetch sales data from the designated source. [TOOL: fetch(source="sales_data")]
**Step 2**: Calculate the total revenue using the fetched sales data.  Assume the 'sales_data' contains a field named 'price' and a field named 'quantity'. The expression calculates the sum of (price * quantity). [TOOL: calc(expr="SUM(price * quantity)")]
**Step 3**: Generate a report indicating the total revenue calculated. [TOOL: report(msg="Total revenue calculated")]



🎯 Task: Analyze customer data

📚 Found 1 similar tasks

📋 Plan:
Okay, here's a step-by-step plan to analyze customer data, using the specified tools and their parameter names:

**Plan:**

Step 1: Retrieve the customer data from the specified source. [TOOL: fetch(source="customer_database")]
Step 2: Perfor